In [1]:
pip install pytorch_msssim

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
unzip MSG-MFGAN.zip

SyntaxError: invalid character in identifier (2608596870.py, line 1)

In [1]:
unzip -d /root/autodl-tmp multi_focus.zip

SyntaxError: invalid syntax (438976797.py, line 1)

In [ ]:
zip MSG-MFGAN.zip MSG-MFGAN

In [2]:
import torch
import torch.nn as nn
import torch as th
from Gan import *
import torch.nn as nn
from data_loader import data_loader
from torch.utils.data import Dataset, DataLoader
import os
torch.cuda.empty_cache()
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"


# torch.cuda.set_per_process_memory_fraction(0.5, device=None)

class ARGS:
    def __init__(self):
        self.datapath="autodl-tmp/multi_focus"
        self.init_size=4
        self.final_size=256
        self.device=th.device("cuda:0" if th.cuda.is_available() else "cpu")
        self.adam_beta1=0.9
        self.adam_beta2=0.999
        self.g_lr=0.0001
        self.d_lr=0.0004
        self.generator_optim_file=None
        self.discriminator_optim_file=None
        self.generator_save_path=None
        self.discriminator_save_path=None
        self.loss_function="RelativeplusContentLoss"
        self.alpha=6
        self.beta=1
        self.num_epochs=200 #
        self.checkpoint_factor=4#
        self.feedback_factor=8#
        self.data_percentage=100
        self.num_samples=8#
        self.sample_dir="tf-logs/recordings/images"
        self.batch_size=16 #52
        self.start=1
        self.loss_dir="tf-logs/recordings/loss"
        self.model_dir="tf-logs/recordings/model"
        
args=ARGS()        
datapath=args.datapath
init_size=args.init_size
final_size=args.final_size
device=args.device

#get the data
data=data_loader.MultiFocusDataset(datapath,mode="train",size=final_size)
data=DataLoader(data, batch_size=args.batch_size, shuffle=True)
testdata=data_loader.MultiFocusDataset(datapath,mode="test",size=final_size)
testdata=DataLoader(testdata, batch_size=args.num_samples, shuffle=True)
#get the MSG_MFGAN model
msg_mfgan=Gan.MSG_MFGAN(init_size=init_size,final_size=final_size,device=device)

gen_optim = th.optim.Adam(msg_mfgan.gen.parameters(), args.g_lr,
                              [args.adam_beta1, args.adam_beta2])
dis_optim = th.optim.Adam(msg_mfgan.dis.parameters(), args.d_lr,
                              [args.adam_beta1, args.adam_beta2])

if args.generator_optim_file is not None:
        gen_optim.load_state_dict(th.load(args.generator_optim_file))

if args.discriminator_optim_file is not None:
        dis_optim.load_state_dict(th.load(args.discriminator_optim_file))
if args.generator_save_path is not None :
        msg_mfgan.gen.load_state_dict(th.load(args.generator_save_path))
if args.discriminator_save_path is not None :
        msg_mfgan.dis.load_state_dict(th.load(args.discriminator_save_path))
    
        
loss_name = args.loss_function.lower()
if loss_name=="relativepluscontentloss":
    loss=Loss.RelativeplusContentLoss(device=args.device,dis=msg_mfgan.dis,alpha=args.alpha,beta=args.beta)

msg_mfgan.train(
        data,
        gen_optim,
        dis_optim,
        loss_fn=loss,
        testdata=testdata,
        num_epochs=args.num_epochs,
        checkpoint_factor=args.checkpoint_factor,
        data_percentage=args.data_percentage,
        feedback_factor=args.feedback_factor,
        num_samples=args.num_samples,
        sample_dir=args.sample_dir,
        save_dir=args.model_dir,
        loss_dir=args.loss_dir,
        start=args.start
    )
# train(self, data, gen_optim, dis_optim, loss_fn,testdata,
#               start=1, num_epochs=12, feedback_factor=10, checkpoint_factor=1,
#               data_percentage=100, num_samples=4,
#               loss_dir='muti_recordings/loss', sample_dir='muti_recordings/images',
#               save_dir="./models"):

        

Starting the training process ... 

Epoch: 1
Elapsed [0:00:03.157146] feedback_count: 0 batch: 1  d_loss: 0.516606  g_loss: 20.543900  ssim_train: 0.732948  ssim_test: 0.774040


KeyboardInterrupt: 